In [1]:
import sys
import os
import torch

# 将src目录添加到Python的搜索路径中，这样我们才能导入我们自己写的模块
# 这是一种常见的做法，用于让notebook能够找到项目中的.py文件
# 我们需要返回上级目录'../'才能找到'src'
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.dataset import SentimentDataset, create_data_loader
from transformers import DistilBertTokenizer

print("模块导入成功！")

/opt/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


模块导入成功！


In [2]:
# --- 定义基本参数 ---
VALIDATION_FILE = '../data/processed/validation.csv'
MAX_LEN = 128 # 序列最大长度，与dataset.py中保持一致
TOKENIZER = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# --- 创建 SentimentDataset 的实例 ---
# 这会调用 SentimentDataset 类的 __init__ 方法
print("正在创建 SentimentDataset 实例...")
val_dataset = SentimentDataset(
    file_path=VALIDATION_FILE,
    tokenizer=TOKENIZER,
    max_len=MAX_LEN
)
print("实例创建成功！")

正在创建 SentimentDataset 实例...
实例创建成功！


In [3]:
# 调用 __len__ 方法
dataset_length = len(val_dataset)

print(f"1. 调用 __len__ 方法:")
print(f"   └── 返回的数据集总长度为: {dataset_length} 条")

1. 调用 __len__ 方法:
   └── 返回的数据集总长度为: 4990 条


In [4]:
# 选择一个样本进行查看，例如第10个样本（索引为9）
sample_index = 9

# 调用 __getitem__ 方法
single_item = val_dataset[sample_index]

print(f"2. 调用 __getitem__({sample_index}) 方法:")
print(f"   └── 返回的是一个字典，包含以下键: {list(single_item.keys())}")
print("-" * 30)

# 逐一打印字典中的内容
print(f"   - 'text': (原始文本)")
print(f"     └── \"{single_item['text']}\"")
print("-" * 30)

print(f"   - 'labels': (情感标签，已经转换为PyTorch张量)")
print(f"     └── {single_item['labels']} (类型: {single_item['labels'].dtype})")
print("-" * 30)

print(f"   - 'input_ids': (文本被分词并转换为数字ID后的张量)")
print(f"     └── 形状: {single_item['input_ids'].shape}")
print(f"     └── 内容: {single_item['input_ids']}")
print("-" * 30)

print(f"   - 'attention_mask': (注意力掩码，1代表真实词元，0代表填充词元)")
print(f"     └── 形状: {single_item['attention_mask'].shape}")
print(f"     └── 内容: {single_item['attention_mask']}")
print("-" * 30)


# 为了更直观地理解，我们将 input_ids 转换回词元
tokens = TOKENIZER.convert_ids_to_tokens(single_item['input_ids'])
print("   - 'input_ids' 反向转换为词元:")
print(f"     └── {tokens}")

2. 调用 __getitem__(9) 方法:
   └── 返回的是一个字典，包含以下键: ['text', 'input_ids', 'attention_mask', 'labels']
------------------------------
   - 'text': (原始文本)
     └── "a man that has never lied to a woman has no respect for her feelings"
------------------------------
   - 'labels': (情感标签，已经转换为PyTorch张量)
     └── 2 (类型: torch.int64)
------------------------------
   - 'input_ids': (文本被分词并转换为数字ID后的张量)
     └── 形状: torch.Size([128])
     └── 内容: tensor([ 101, 1037, 2158, 2008, 2038, 2196, 9828, 2000, 1037, 2450, 2038, 2053,
        4847, 2005, 2014, 5346,  102,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
 

In [5]:
# 定义批大小
BATCH_SIZE = 4

# 调用我们写的辅助函数来创建 DataLoader
val_data_loader = create_data_loader(
    file_path=VALIDATION_FILE,
    tokenizer=TOKENIZER,
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE
)

# 从 DataLoader 中取出一个批次的数据
first_batch = next(iter(val_data_loader))


print("3. 从 DataLoader 中获取一个批次的数据:")
print(f"   └── DataLoader 返回的也是一个字典，包含以下键: {list(first_batch.keys())}")
print("-" * 30)

# 打印这个批次中各个张量的形状
# 注意，形状的第一维现在是 BATCH_SIZE (4)
print(f"   - 'input_ids' 的形状: {first_batch['input_ids'].shape}")
print(f"     └── [批大小, 最大长度] -> [{BATCH_SIZE}, {MAX_LEN}]")
print("")
print(f"   - 'attention_mask' 的形状: {first_batch['attention_mask'].shape}")
print(f"     └── [批大小, 最大长度] -> [{BATCH_SIZE}, {MAX_LEN}]")
print("")
print(f"   - 'labels' 的形状: {first_batch['labels'].shape}")
print(f"     └── [批大小] -> [{BATCH_SIZE}]")

3. 从 DataLoader 中获取一个批次的数据:
   └── DataLoader 返回的也是一个字典，包含以下键: ['text', 'input_ids', 'attention_mask', 'labels']
------------------------------
   - 'input_ids' 的形状: torch.Size([4, 128])
     └── [批大小, 最大长度] -> [4, 128]

   - 'attention_mask' 的形状: torch.Size([4, 128])
     └── [批大小, 最大长度] -> [4, 128]

   - 'labels' 的形状: torch.Size([4])
     └── [批大小] -> [4]
